# Hands On Astropy Tutorial - Solutions to the exercises
**Tutors:** Axel Donath and Christoph Deil


This notebook contains the solutions to the exercises in [astropy_hands_on.ipynb](https://github.com/Asterics2020-Obelics/School2017/blob/master/astropy/astropy_hands_on.ipynb)

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## 1. Units and Quantities

- (*easy*) How long does the light travel from the sun to the earth in minutes? How long does the light travel from the Galactic center (assume a distance of 8 kpc) in years? 


In [2]:
from astropy import constants as const
import astropy.units as u

# distance sun - earth
distance_sun = 1 * u.au
# speed of light
speed = const.c
# time
time_sun = distance_sun / speed
print("Light travels from sun to earth in {:.2f}".format(time_sun.to('min')))

# distance to GC
distance_gc = 8 * u.kpc
# time
time_gc = distance_gc / speed
print("Light travels from the GC to earth in {:.2f}".format(time_gc.to('yr')))

Light travels from sun to earth in 8.32 min
Light travels from the GC to earth in 26092.51 yr


- (*advanced*) Define a new unit called `"baro-meter"`, which is eqivalent to 25 cm and use it to measure the height of the empire state building (assume a height of 381 meters). Please read the [Astropy documentation on equivalencies](http://docs.astropy.org/en/stable/units/equivalencies.html#writing-new-equivalencies) for an example how to do this ( For other ways to measure the height of a building using a barometer see [barometer question on Wikipedia](https://en.wikipedia.org/wiki/Barometer_question)...)

In [3]:
# define new unit baro-meter
bm = u.def_unit('baro-meter')

# define equivalency
baro_meter = [
   (bm, u.cm, lambda x: 25 * x, lambda x: x / 25)
]

# height of the empire state building
height = 381 * u.m

# convert to baro-meters
height_bm = height.to(bm, equivalencies=baro_meter)
print("The height of the empire state building in baro-meters is {}".format(height_bm))

The height of the empire state building in baro-meters is 1524.0 baro-meter


- (*expert*) Find the frequency with the maximum photon rate of the black body spectrum $\mathcal{B}(\nu, T)$ with temperature $T=5000~\mathrm{K}$. For this you can use a numerical optimization routine such as [scipy.optimize.minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html).
As the frequency as well as the radiance can cover many orders of magnitude, the stability of the algorithm can be much improved by minimizing the (negative) logarithm of the radiance $-\log{\mathcal{B}}$ as well as minimizing with respect to the logarithm of the frequency by introducing the variable $x = \log{\nu}$. Finally compare the result against the analytical solution from [Wien's displacement law](https://en.wikipedia.org/wiki/Wien%27s_displacement_law). 

In [4]:
from scipy.optimize import minimize

# Copy over the definition from the tutorial notebook 
@u.quantity_input(frequency=u.hertz, temperature=u.K)
def blackbody(frequency, temperature): 
    pre_factor = 2 * (const.h * frequency ** 3) / const.c ** 2
    exponential_factor = 1. / (np.exp((const.h * frequency) / (const.k_B * temperature)) - 1)
    return pre_factor * exponential_factor


# Define the objective function that shall be minimized
# We introduce x = np.log(frequency) => frequency = np.exp(x)
# And convert the radiance to well defined units and take the negative logarithm
def objective_funtion(x):
    frequency = np.exp(x) * u.Hz
    radiance = blackbody(frequency=frequency, temperature=5000 * u.K)
    value = -np.log(radiance.to('Hz3 J s3 / m2').value)
    return value
 
result = minimize(objective_funtion, x0=np.log(1E14), method='Nelder-Mead') 
frequency_max = np.exp(result['x']) * u.Hz

# Equation taken from https://en.wikipedia.org/wiki/Wien%27s_displacement_law#Alternate_Maxima
@u.quantity_input(temperature=u.K)
def wiens_displacement_law(temperature):
    return 0.058789e12 * u.Hz / u.K * temperature 


frequency_max_wien = wiens_displacement_law(temperature=5000*u.K)

In [ ]:
print("The numerical optimization gives: {[0]:.3e}".format(frequency_max))
print("Wien's displacement law gives   : {:.3e}".format(frequency_max_wien))

The numerical optimization gives: 2.939e+14 Hz
Wien's displacement law gives   : 2.939e+14 Hz


## 2. Coordinates

- (*easy*) Define the sky coordinate for your favorite object and find the distance to the crab nebula and Galactic center.

In [ ]:
from astropy.coordinates import SkyCoord

# Coordinate of PKS2155-305
coord_pks = SkyCoord.from_name('pks2155-304')

# Coordinate of the Crab Nebula
coord_crab = SkyCoord.from_name('crab')
separation_crab = coord_pks.separation(coord_crab)
print('The angular distance between PKS2155-204 and the Crab Nebula is {:.2f}'.format(separation_crab))

# Coordinate of the Galactic Center
coord_gc = SkyCoord(0, 0, unit='deg', frame='galactic')
separation_gc = coord_pks.separation(coord_gc)
print('The angular distance between PKS2155-204 and the Galactic Center is {:.2f}'.format(separation_gc))

- (*expert*) Make a plot of the height above horizon vs.time for the crab position at the location of Annecy. Mark the the time range where it is visible. Would the crab be visible tonight?

In [ ]:
from astropy.coordinates import EarthLocation, AltAz
from astropy.time import Time
import numpy as np
from astropy.visualization import quantity_support

# Define coordinate
coord_crab = SkyCoord.from_name('crab')

# Set location
annecy = EarthLocation(lat=45.8058 * u.deg, lon=6.5726 * u.deg)

# Define array of times (30 minute intervals)
time_intervals = np.linspace(0 * u.day, 1 *u.day, 48)

time_start = Time('2017-06-09 00:00:00')
times = Time(time_start + time_intervals)

# Get AltAz coordinates
altaz = AltAz(obstime=times, location=annecy)
crab_altaz = coord_crab.transform_to(altaz)

# Get altitude over horizon
altitudes = crab_altaz.alt.to('deg')

# Get times when altitude > 0
above_horizon = altitudes > 0 * u.deg

with quantity_support():
    plt.plot_date(times.plot_date, altitudes, fmt='-')
    
    # orient date labels at a slant
    plt.gcf().autofmt_xdate() 
    
    alt_min, alt_max = -25 * u.deg, 75 * u.deg
    plt.fill_between(times.plot_date,
                     alt_min, alt_max,
                     where=above_horizon,
                     facecolor='green',
                     alpha=0.5)
    plt.ylim(alt_min, alt_max)

## 3. Tables

- (*easy*) Add columns with the `RA` and `DEC` coordinates of the objects to the example table.

In [ ]:
# Get example table
from astropy.table import Table
table = Table()
table['Source_Name'] = ['Crab', 'Sag A*', 'Cas A']
table['GLON'] = [184.55754381, 0, 111.74169477] * u.deg
table['GLAT'] = [-5.78427369, 0, -2.13544151] * u.deg

# Create SkyCoord object holding all 3 sky coordinates
coords = SkyCoord(table['GLON'].quantity, table['GLAT'].quantity, frame='galactic')

# Add new columns
table['RA'] = coords.icrs.ra
table['DEC'] = coords.icrs.dec

# Pretty-print the table
table.pprint()

- (*advanced *) Load the Fermi 2FHL catalog table (`'data/fermi_2fhl_catalog.fits'`) and find the brighest source (the brightness value is stored in the `'Flux50'` column). What is the common name of the source (column `'ASSOC'`)?
Find all sources within 10 deg from the position of this source.

In [ ]:
# Read table
fermi_2fhl = Table.read('data/fermi_2fhl_catalog.fits', hdu=1)

# Sort by brightness
fermi_2fhl.sort('Flux50')

# With -1 we get the last element
print('The brightest source in the catalog is: {} \n'.format(fermi_2fhl[-1]['ASSOC']))

# Define crab position
coord_crab = SkyCoord.from_name('Crab') #TODo Define coord from table

# Get SkyCoords of objects in the table
coords = SkyCoord(fermi_2fhl['GLON'], fermi_2fhl['GLAT'], frame='galactic')

# Find rows where separation < 1 deg
separation = coord_crab.separation(coords)

# Print result
max_separation = 10 * u.deg
fermi_2fhl[separation < max_separation]

- (*expert*) Make an allsky plot (using an 'Aitoff' projection) of all the sources in the Fermi 2FHL catalog grouped by source class (i.e. choose a different marker color for every source class). The class of the sources is stored in the `'CLASS'` column. There are a few possible ways to identify the groups of source class. One of them is to use [Table.group_by()](http://docs.astropy.org/en/stable/api/astropy.table.Table.html#astropy.table.Table.group_by) followed by a loop over the groups (check Astropy documentation!) another way is to use a regular Python `set` to idenfity the unique entries in the `'CLASS'` column and loop over those. You can use `plt.subplot(projection='aitoff')` to instanciate a matplotlib axes with `'Aitoff'` projection. Note that the `'Aitoff'` projection requires angular positions measured in radians.

In [ ]:
source_classes = set(fermi_2fhl['CLASS'])

print('Source classes: {}'.format(source_classes))

In [ ]:
from astropy.coordinates import Angle

plt.figure(figsize=(12, 5))
ax = plt.subplot(projection='aitoff')

glon = Angle(fermi_2fhl['GLON'].quantity)
glon = glon.wrap_at('180d')

glat = Angle(fermi_2fhl['GLAT'].quantity)

with quantity_support():
    for source_class in sorted(source_classes):
        selection = fermi_2fhl['CLASS'] == source_class
        ax.scatter(glon[selection].rad, glat[selection].rad, label=source_class)
    ax.grid()
    plt.legend(bbox_to_anchor=(1.15, 1), loc=1)

## 4. FITS Images and WCS

- (*easy*) Read in the data from the other energy bands (`'data/casa_1.5-3.0keV.fits.gz'` and `'data/casa_4.0-6.0keV.fits.gz'`) as well, sum up all the data from the three energy bands. Plot the summed data using [plt.imshow()](https://matplotlib.org/devdocs/api/_as_gen/matplotlib.axes.Axes.imshow.html) again.

In [ ]:
from astropy.io import fits

filenames = ['data/casa_0.5-1.5keV.fits.gz', 'data/casa_1.5-3.0keV.fits.gz',
             'data/casa_4.0-6.0keV.fits.gz']

data_list = []

for filename in filenames:
    image_hdu = fits.open(filename)['PRIMARY']
    data_list.append(image_hdu.data)
    
data_summed = np.sum(data_list, axis=0)
plt.imshow(data_summed, origin='lower', cmap='afmhot')

- (*advanced*) Cutout the central region of the image using [astropy.nddata.Cutout2D](http://docs.astropy.org/en/stable/api/astropy.nddata.Cutout2D.html) (size of 0.05 deg x 0.05 deg around the position of `'Cas A'`). Find the sky position of the brightest pixel in the cutout data. For this [np.argmax()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmax.html) and [np.unravel_index()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.unravel_index.html) might be helpful. Make sure you use the updated `.data` and `.wcs` attributes of the `Cutout2D` object to achieve the latter.

In [ ]:
from astropy.nddata.utils import Cutout2D
from astropy.wcs import WCS

image_hdu = fits.open('data/casa_0.5-1.5keV.fits.gz')['PRIMARY']
wcs = WCS(image_hdu.header) 
image_data = image_hdu.data

pos_casa = SkyCoord.from_name('Cas A')

data_cutout = Cutout2D(image_data, position=pos_casa,
                       wcs=wcs, size=0.05 * u.deg)

# Find index of maximum value (this will return the
# index on the flattened array)
idx_flat = data_cutout.data.argmax()

# Transform back to the original shape
idx = np.unravel_index(idx_flat, data_cutout.data.shape)

# Transform to SkyCoord
pos = SkyCoord.from_pixel(*idx, wcs=data_cutout.wcs)
print("The position of the brightest pixel in the image is: {}".format(pos))

- (*very hard*) Calculate a radial profile of the image data to estimate the size of the shell of the supernova remnant. For this sum up all the data within a set of rings (specified  by `radii_min` and `radii_max`, choose a total minimum radius of `0 arcmin` and total maximum radius of `5 arcmin`) around the center position of *Cas A*. You can use [np.indices()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.indices.html) to get arrays of x and y pixel positions. Make sure you use clever numpy broadcasting to avoid a Python loop over the `radii_min` and `radii_max` arrays (**hint**: [ndarray.reshape(-1, ...)](https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html#numpy.reshape) might be useful to expand the data and radius array to the third dimension).

In [ ]:
from astropy.wcs import WCS
 
# open image
image_hdu = fits.open('data/casa_1.5-3.0keV.fits.gz')['PRIMARY']
wcs = WCS(image_hdu.header)
image_data = image_hdu.data

# define position and integration radius
pos_casa = SkyCoord.from_name('Cas A')

# create an array of sky positions for each pixel
yp, xp = np.indices(image_data.shape)
sky_positions = SkyCoord.from_pixel(xp=xp, yp=yp, wcs=wcs)

# calculate separation image
separation = pos_casa.separation(sky_positions)

# initialize radius arrays
radii = np.linspace(0, 5, 50).reshape(-1, 1, 1) * u.arcmin
radii_min, radii_max = radii[:-1], radii[1:]

# mask out the annulus regions
mask = (separation > radii_min) & (separation < radii_max)

data_masked = mask * image_data.reshape(-1, 1024, 1024)
data_summed = data_masked.sum(axis=-1).sum(axis=-1)

radius_center = (radii_min[:, 0, 0] + radii_max[:, 0, 0]) / 2.

with quantity_support():
    plt.plot(radius_center, data_summed)
    plt.xlabel('Radius (arcmin)')
    plt.ylabel('Profile Amplitude (A.U.)')


**Note:** Even if we avoided the Python loop by expanding the Numpy data array in the third dimension, this is still not
the most efficient implementation to measure radial profiles on images. Better solutions are based on counting pixels by indices. E.g. look at the implementations http://docs.gammapy.org/dev/_modules/gammapy/image/radial_profile.html#radial_profile or https://stackoverflow.com/questions/21242011/most-efficient-way-to-calculate-radial-profile/21242776#2124277e.

## 5. Plotting of sky images (optional)

- (*easy*) Make a combined plot of the Chandra data for all energy bands side by side (one row, three columns)
and put the energy information in the title of the subplot. You can use e.g. `plt.sublots()`.  

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS

# We define a figure size, that has an aspect ratio to accomodate three plots side by side
fig = plt.figure(figsize=(12, 3))
filenames = ['data/casa_0.5-1.5keV.fits.gz', 'data/casa_1.5-3.0keV.fits.gz', 'data/casa_4.0-6.0keV.fits.gz']

# For convenience we use a Python loop here, but the same can be achieved
# by copy and pasting the code for one energy band
for idx, filename in enumerate(filenames):
    image_hdu = fits.open(filename)['PRIMARY']
    wcs = WCS(image_hdu.header)
    ax = plt.subplot(1, 3, idx + 1, projection=wcs)
    ax.imshow(image_hdu.data, origin='lower', cmap='afmhot')
    
    # We extract the energy range from the filename, by setting it "by hand" is also fine
    energy_range = filename[10:20]
    ax.set_title('Energy range: {}'.format(energy_range))
    
    ax.set_xlabel('RA (deg)')
    ax.set_ylabel('DEC (deg)')
    

- (*advanced*) Make a combined RGB image of the Chandra data in different energy bands. Therefore you have to
renormalize the data per energy band to the maximum value of respective energy band and combine the data into a 3d array with a shape that is recognised by `plt.imshow()` (check the matplotlib documentation!).

In [ ]:
from astropy.visualization import AsinhStretch

# use asinh stretching to brighten up the image
stretch = AsinhStretch(0.1)

data_rgb = []

for filename in filenames:
    image_hdu = fits.open(filename)['PRIMARY']
    data = image_hdu.data
    data /= data.max()
    data = stretch(data)
    data_rgb.append(data)

data_rgb_stacked = np.stack(data_rgb, axis=2)

plt.figure(figsize=(10, 10))
ax = plt.subplot(projection=wcs)
ax.imshow(data_rgb_stacked, origin='lower')

ax.set_xlabel('RA (deg)')
ax.set_ylabel('DEC (deg)')

## Acknowledgements
![](asterics_log.png)

This tutorial was supported by the H2020-Astronomy ESFRI and Research Infrastructure Cluster (Grant Agreement number: 653477).